In [1]:
from utz import *
from nj_crashes.paths import ROOT_DIR
from njsp import crashes
from njsp.paths import PROJECTED_CSV
from njsp.ytc import to_ytc
from njsp.ytd import oldest_commit_rundate_since, projected_roy_deaths
from njsp import Ytd

chdir(ROOT_DIR)

In [2]:
ytd = Ytd()
prv_rundate = ytd.prv_rundate
prv_rundate

Searching for oldest commit with rundate ≥2024-03-09


Found rundate 2024-03-08 10:00:02 < 2024-03-09 at commit a14ca405; returning commit 4ad65ac


'2024-03-09 11:02:42'

In [3]:
prv_ytd_ratio = ytd.cur_ytd_deaths / ytd.prv_ytd_deaths
pct_change = (prv_ytd_ratio - 1) * 100

print(f'As of {ytd.prv_rundate}, NJSP was reporting {ytd.prv_ytd_total} YTD deaths')
print(f'Current YTD Deaths ({ytd.rundate}): {ytd.cur_ytd_deaths}')
print(f'Previous year YTD Deaths (adjusted; {ytd.prv_rundate}): {ytd.prv_ytd_deaths}')
print(f'Projected {ytd.cur_year} total: {ytd.projected_year_total}')
print(f'{pct_change:.1f}% change')

As of 2024-03-09 11:02:42, NJSP was reporting 107 YTD deaths
Current YTD Deaths (2025-03-09 10:00:02-04:00): 87
Previous year YTD Deaths (adjusted; 2024-03-09 11:02:42): 105.59241181150864
Projected 2025 total: 653.3807146749409
-17.6% change


FAUQStats cache miss: 2024, Sat Mar 09 11:02:42 EST 2024


Find the oldest commit with rundate less than 1 year ago, load crashes from that time:

In [4]:
prv_crashes = ytd.prv_ytd_crashes
prv_crashes

,CCODE,CNAME,MCODE,MNAME,HIGHWAY,LOCATION,FATALITIES,FATAL_D,FATAL_P,FATAL_T,FATAL_B,INJURIES,STREET,dt
ACCID,,,,,,,,,,,,,,
12918,12,Middlesex,1218,Plainsboro Twsp,1,State Highway 1,1.0,0.0,0.0,1.0,0.0,NaN,NaN,2024-01-01 01:56:00-05:00
12923,11,Mercer,1111,Trenton,129,State Highway 129,1.0,1.0,0.0,0.0,0.0,NaN,NaN,2024-01-03 12:14:00-05:00
12932,01,Atlantic,0112,Hamilton Twsp,446,Atlantic City Expressway MP 20.5,1.0,1.0,0.0,0.0,0.0,NaN,NaN,2024-01-05 09:14:00-05:00
12933,02,Bergen,0248,Ramsey Boro,NaN,East Oak St,1.0,1.0,0.0,0.0,0.0,NaN,East Oak St,2024-01-05 14:12:00-05:00
12927,20,Union,2009,Linden City,27,State Highway 27,1.0,0.0,0.0,1.0,0.0,NaN,NaN,2024-01-06 08:37:00-05:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13039,15,Ocean,1507,Toms River,444,Garden State Parkway MP 83.6,1.0,1.0,0.0,0.0,0.0,NaN,NaN,2024-03-03 18:50:00-05:00
13046,12,Middlesex,1214,New Brunswick City,1,State Highway 1,1.0,1.0,0.0,0.0,0.0,NaN,NaN,2024-03-06 07:08:00-05:00
13043,19,Sussex,1907,Fredon Twsp,622,County 622 MP 3.9,1.0,1.0,0.0,0.0,0.0,NaN,NaN,2024-03-06 21:25:00-05:00


Load previous year YTD and final counts, and current year YTD counts:

In [5]:
prv_year = ytd.prv_year
cur_year = ytd.cur_year

prv_ytc = to_ytc(prv_crashes)
prv_ytd = prv_ytc.loc[prv_year]

prv_end = to_ytc(ytd.prv_end_crashes).loc[prv_year]

cur_ytc = to_ytc(ytd.cur_ytd_crashes)
cur_ytd = cur_ytc.loc[cur_year]

cur_ytd

,driver,passenger,pedestrian,cyclist,crashes
county,,,,,
Atlantic,3,0,2,0,5
Bergen,4,1,1,0,6
Burlington,3,0,2,0,5
Camden,8,0,2,0,10
Cape May,2,0,0,0,2
Cumberland,3,1,1,0,4
Essex,0,0,2,0,2
Gloucester,4,0,0,0,4
Hudson,0,0,1,0,1


Compute fraction of current year that has elapsed (year-to-date; "YTD") and still remains (rest-of-year; "ROY"):

In [6]:
cur_ytd_frac = ytd.cur_year_frac
cur_roy_frac = 1 - cur_ytd_frac
print('%.1f%% through the year, %.1f%% remaining' % (ytd.cur_year_frac * 100, ytd.cur_roy_frac * 100))

18.5% through the year, 81.5% remaining


Combine previous year YTD / end with current year YTD:

In [7]:
def melt(df, name):
    return (
        df
        .melt(ignore_index=False, var_name='type')
        .set_index('type', append=True)
        .value
        .rename(name)
    )

z = sxs(
    melt(prv_ytd, 'prv_ytd'),
    melt(prv_end, 'prv_end'),
    melt(cur_ytd, 'cur_ytd'),
).fillna(0).astype(int)
z

prv_ytd  prv_end  cur_ytd
county     type                                 
Atlantic   driver            3       31        3
Bergen     driver            5       14        4
Burlington driver            5       32        3
Camden     driver            4       19        8
Cape May   driver            1        8        2
...                        ...      ...      ...
Somerset   driver            0       11        4
           passenger         0        0        1
           pedestrian        0        8        0
           cyclist           0        0        0
           crashes           0       18        5

[105 rows x 3 columns]

In [8]:
def project_roy(r):
    return int(round(projected_roy_deaths(r.prv_ytd, r.prv_end, r.cur_ytd, cur_ytd_frac)))

roy = z.apply(project_roy, axis=1).rename('roy')
projected = (
    (z.cur_ytd + roy)
    .rename('projected')
    .reset_index(level=1)
    .pivot(columns='type', values='projected')
)
projected

type,crashes,cyclist,driver,passenger,pedestrian
county,,,,,
Atlantic,47,2,31,1,17
Bergen,33,1,13,5,15
Burlington,42,1,28,2,11
Camden,40,0,26,3,11
Cape May,11,0,10,0,1
Cumberland,34,0,28,7,7
Essex,41,0,15,7,24
Gloucester,29,0,20,6,5
Hudson,15,1,7,1,7


In [9]:
projected.to_csv(PROJECTED_CSV)